# Testing the correction protocol

In [84]:
import pyqsp
from pyqsp.gadgets import *
import numpy as np
import pennylane as qml

np.set_printoptions(linewidth=100000)

In [85]:
n = 24

L = ExtractionGadget(n, 'G').get_sequence(('G', 0))
L_prime = ExtractionGadget(n, 'G').get_sequence(('G', 0))

Phi, Phi_prime = [], []

for op in L:
    if isinstance(op, QSP_Rotation):
        Phi.append(op.theta)
        
for op in L_prime:
    if isinstance(op, QSP_Signal):
        Phi_prime.append(op)

In [86]:
Phi = Phi[::-1]
Phi_prime = Phi_prime[::-1]

In [87]:
len(Phi_prime)

25

In [88]:
# Testing the protocol!

dev = qml.device('default.qubit', wires=[0, 1, 2])

def V():
    qml.CSWAP(wires=[0, 1, 2])
    qml.RX(0.4, wires=2)
    qml.CSWAP(wires=[0, 1, 2])

@qml.qnode(dev)
def func():
    for p in range(len(Phi[0:len(Phi)-2])):
        qml.RZ(Phi[p], wires=2)

In [89]:
dev = qml.device('default.qubit', wires=[0, 1, 2])

@qml.qnode(dev)
def func():
    qml.RZ(-0.8, wires=2)
    qml.CSWAP(wires=[0, 1, 2])
    qml.RZ(0.8, wires=1)
    qml.CSWAP(wires=[0, 1, 2])
    return qml.state()

In [130]:
qml.matrix(func)()[2][2]

(0.6967067093471654+0.7173560908995228j)

In [129]:
np.cos(0.4)

0.9210609940028851

In [37]:
Corrective_CSWAP(1).matrix([1])

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [38]:
# Constructs an atomic gadgets
Xi_1 = np.array([[0, 0]])
S_1 = [[0]]
G1 = AtomicGadget(Xi_1, S_1, label="G1")

Xi_2 = np.array([[1, 2, -2, -1]])
S_2 = [[0, 1, 0]]
G2 = AtomicGadget(Xi_2, S_2, label="G2")

In [39]:
# Constructs the interlink between the gadgets
G = G1.interlink(G2, [(('G1', 0), ('G2', 0), 20)])

In [40]:
len(G1.get_sequence(('G1', 0), correction=8))

40

In [41]:
# Gets the QSP unitary
U = lambda x : G1.get_qsp_unitary(('G1', 0), correction=20, rot={('G1', 0):0.4})( {('G1', 0) : x})

In [42]:
U(0.3) @ np.kron(np.kron(np.array([1, 0]), np.array([0, 1])), np.array([1, 0]))

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, -6.96706709e-01+7.17356091e-01j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j, -7.53257564e-07-7.31480981e-07j,  0.00000000e+00+0.00000000e+00j])

In [43]:
np.sin(0.8)

0.7173560908995228

In [20]:
np.cos(0.4) + 1j * np.sin(0.4)

(0.9210609940028851+0.3894183423086505j)

In [21]:
U(0.3)[3][0]

0j

In [22]:
np.kron(np.array([[0, 1], [1, 0]]), np.eye(2)) @ np.array([0, 1, 0, 0])

array([0., 0., 0., 1.])

In [23]:
(np.cos(0.8) + 1j * np.sin(0.8)) * (1j * np.sqrt(1 - 0.1 ** 2))

(-0.7137602983769397+0.6932144231483228j)

In [24]:
(1j * np.sqrt(1 - 0.1 ** 2))

0.99498743710662j